In [4]:
import os
import arcpy

Setting workspace to be my project GDB

In [41]:
# workspace
arcpy.env.workspace = r'C:\Users\dani9510\Documents\ArcGIS\Projects\Utah Accidents Demo\Utah Accidents Demo.gdb'
arcpy.env.overwriteOutput = True

First we need to parse the crashes CSV file. 

Update this path to wherever the csv is stored for you

In [42]:
collisions_csv = r'C:/Users/dani9510/Documents/Projects/Vehicle Accident Prediction/Utah crashes_2010-2017.csv'
res = arcpy.TableToTable_conversion(collisions_csv,arcpy.env.workspace,'collisions')

This needs to be turned into a layer to do further GP

In [47]:
collisions_csv = r'C:/Users/dani9510/Documents/Projects/Vehicle Accident Prediction/Utah crashes_2010-2017.csv'
in_sr = arcpy.SpatialReference(4269)
res = arcpy.MakeXYEventLayer_management('collisions',
        in_x_field='Longitude',
        in_y_field='Latitude',
        out_layer='collisions_with_xy',
        spatial_reference=in_sr)

Since the data is actually in UTM, we need to add a field to process

In [44]:
res = arcpy.AddField_management(
        in_table='collisions_with_xy',
        field_name='UTM',
        field_type='String')

Next we need to calculate the UTM field from the x/y fields

In [45]:
expression ="utm(!Longitude!,!Latitude!)"
code_block =\
"""
def utm(x,y):
    try:
        return "12N{}{}".format(int(x),int(y))
    except:
        return None
"""
res = arcpy.CalculateField_management(
        in_table='collisions_with_xy',
        field='UTM',
        expression=expression,
        code_block=code_block)

Finally, convert the UTM coordinates to WGS84 decimal lat/lon

In [46]:
res = arcpy.ConvertCoordinateNotation_management(
        in_table='collisions_with_xy',
        out_featureclass='collisions_with_xy_converted',
        x_field='UTM',
        input_coordinate_format='UTM_ZONES')